Imports - импортируем все необходимое

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
X_full = pd.read_csv('db.csv', index_col='id')
X_test_full = pd.read_csv('db.csv', index_col='id')

In [3]:
X_full

,giving_dttm,enrollment_dttm,down_time_train,facts_departures_id,transportation_sheet_id,interval_giving,departure_dttm,end_pending_dttm,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time
id,,,,,,,,,,,,,
24,2021-10-05 04:01:00,2021-10-05 04:36:00,0.583333,34,920,0.0,2021-10-05 11:37:00,2021-10-05 06:40:00,4.950000,-7.600000,2.40000,-2.050008,NaN
32,2021-10-01 07:10:00,2021-10-01 09:10:10,2.002778,14,778,0.0,2021-10-02 00:50:00,2021-09-30 22:50:00,26.000000,-17.666667,NaN,NaN,NaN
33,2021-10-01 05:00:00,2021-10-01 07:40:00,2.666667,22,763,0.0,2021-10-01 02:02:00,2021-09-30 23:45:00,2.283333,2.966667,NaN,NaN,NaN
34,2021-10-06 17:30:00,2021-10-06 19:30:00,2.000000,43,939,0.0,2021-10-06 14:02:00,2021-10-06 07:30:00,6.533333,3.466667,NaN,0.900000,NaN
35,2021-10-06 19:20:00,2021-10-06 22:20:00,3.000000,44,941,0.0,2021-10-06 16:09:00,2021-10-06 12:30:00,3.650000,3.183333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5394,2022-04-30 17:20:00,2022-04-30 18:05:00,0.750000,5372,16838,0.0,2022-04-30 14:00:00,2022-04-30 12:30:00,1.500000,3.333333,NaN,NaN,0.433333
5395,2022-04-30 18:05:00,2022-04-30 20:30:00,2.416667,5371,16839,0.0,2022-04-30 15:24:00,2022-04-30 09:40:00,5.733333,2.683333,NaN,NaN,NaN
5396,2022-04-30 20:45:00,2022-04-30 21:15:00,0.500000,5344,16749,0.0,2022-04-29 08:51:00,2022-04-29 06:30:00,2.350000,35.900000,5.15004,1.116672,4.266667


Data explorationData exploration

In [4]:
print(X_full.columns)

Index(['giving_dttm', 'enrollment_dttm', 'down_time_train',
       'facts_departures_id', 'transportation_sheet_id', 'interval_giving',
       'departure_dttm', 'end_pending_dttm', 'idle_train', 'travel_time',
       'crew_down_time', 'crew_down_time_plan', 'intermediate_down_time'],
      dtype='object')


In [5]:
X_full.sample(5)

,giving_dttm,enrollment_dttm,down_time_train,facts_departures_id,transportation_sheet_id,interval_giving,departure_dttm,end_pending_dttm,idle_train,travel_time,crew_down_time,crew_down_time_plan,intermediate_down_time
id,,,,,,,,,,,,,
14655,2023-05-02 07:39:00,2023-05-02 13:35:00,5.933333,14679,52960,0.0,2023-05-02 02:38:00,2023-05-02 01:00:00,1.633333,5.016667,NaN,NaN,1.700000
22424,2024-03-11 14:36:00,2024-03-11 17:00:00,2.400000,22477,83587,0.0,2024-03-11 11:29:00,2024-03-11 07:35:00,3.900000,3.116667,NaN,NaN,0.266667
31407,2025-02-16 21:49:00,2025-02-16 22:20:00,0.516667,31517,118940,0.0,2025-02-16 18:10:00,2025-02-16 14:20:00,3.833333,3.650000,NaN,NaN,0.100000
31214,2025-02-08 19:42:00,2025-02-08 20:12:00,0.500000,31324,118219,0.0,2025-02-08 16:40:00,2025-02-08 14:00:00,2.666667,3.033333,NaN,NaN,1.216667
22436,2024-03-12 02:03:00,2024-03-12 03:10:00,1.116667,22488,83627,0.0,2024-03-11 22:43:00,2024-03-11 19:30:00,3.216667,3.333333,NaN,NaN,NaN


In [6]:
X_full.describe(include='object')

,giving_dttm,enrollment_dttm,departure_dttm,end_pending_dttm
count,33302,33302,33302,33302
unique,32803,32187,32907,32049
top,2024-07-24 17:42:00,2021-10-03 13:10:00,2022-06-29 01:35:00,2021-10-24 16:50:00
freq,3,4,3,3


In [7]:
X_full.drop(['facts_departures_id'], axis=1, inplace=True)
X_full.drop(['transportation_sheet_id'], axis=1, inplace=True)
X_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33302 entries, 24 to 5969
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   giving_dttm             33302 non-null  object 
 1   enrollment_dttm         33302 non-null  object 
 2   down_time_train         33302 non-null  float64
 3   interval_giving         33302 non-null  float64
 4   departure_dttm          33302 non-null  object 
 5   end_pending_dttm        33302 non-null  object 
 6   idle_train              33302 non-null  float64
 7   travel_time             33302 non-null  float64
 8   crew_down_time          11797 non-null  float64
 9   crew_down_time_plan     3866 non-null   float64
 10  intermediate_down_time  19889 non-null  float64
dtypes: float64(7), object(4)
memory usage: 3.0+ MB


Data preparation - осуществляем базовые преобразования датасета

In [8]:
# Удаляем из датасета строки, в которых недостаточно данных для обучения.
X_full.dropna(axis=0, subset=['intermediate_down_time'], inplace=True)
# Разбиваем датасет на цель (y) и входные данные (X)
y = X_full.intermediate_down_time
X_full.drop(['intermediate_down_time'], axis=1, inplace=True)

# Разбиваем датасет на train/val
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [9]:
# В наших данных есть нечисловые значения. Они затрудняют обработку
# поэтому мы оставим только те данные, где нечисловые значения имеют меньше 10 разных значений
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                    X_train_full[cname].dtype == "object"]

# и все числовые данные
numerical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].dtype in ['int64', 'float64']]

# все столбцы, в которых количество пропусков больше 10% X[cname].isnull().sum()

# оставим в датасете только те колонки, с которыми хотим работать дальше
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

Sanity check

In [10]:
print(len(X_train.columns), len(X_train_full.columns))
X_train.head()

6 10


,down_time_train,interval_giving,idle_train,travel_time,crew_down_time,crew_down_time_plan
id,,,,,,
29594,2.266667,0.0,7.883333,4.433333,NaN,NaN
29038,4.250000,0.0,5.616667,2.950000,NaN,NaN
24935,29.300000,0.0,2.650000,33.633333,7.03332,NaN
30012,3.316667,1.0,1.383333,3.300000,NaN,NaN
13804,0.000000,0.0,1.500000,3.000000,NaN,NaN


#Pipeline

Baseline - базовый пайплайн, от которого мы будем отталкиваться в дальнейшем

###Data preparation (preprocessing)

In [11]:
# Инициализируем препроцессор для заполнения недостающих числовых значений
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Инициализируем препроцессор для заполнения недостающих НЕ числовых значений
# Добавляем шаг кодирования НЕ числовых значений в числовые
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Создаем общий препроцессор данных, соединив первые два
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ])

### Model

In [12]:
# Инициализируем модель
model = RandomForestRegressor(n_estimators=100, random_state=0, verbose=True)

In [13]:
# Создаем пайплайн из модели и препроцессора
classifier = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)],
                      verbose=True)

### Training

In [14]:
# Передаем обучающие данные в пайплайн, обучаем
classifier.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.0s


[Pipeline] ............. (step 2 of 2) Processing model, total=   4.5s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.4s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['down_time_train',
                                                   'interval_giving',
                                                   'idle_train', 'travel_time',
                                                   'crew_down_time',
                                                   'crew_down_time_plan']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', RandomForestRegressor(random_state=0, verbose=True))],
         verbose=True)

### Model evaluation

In [15]:
# Передаем валидационные данные в пайплайн, получаем предсказания
preds = classifier.predict(X_valid)

# Оцениваем точность модели
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 0.7430521616013381


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [16]:
y_valid.mean()

np.float64(1.577752638511815)

In [17]:
(1 - 0.7430521616013381/1.577752638511815)*100

52.904394297054836

In [18]:
print('MSE:', mean_squared_error(y_valid, preds))

MSE: 1.8556260188171068


In [19]:
routes_to_check = 80
print(X_valid.iloc[routes_to_check].to_string())
print()
print(y_valid.iloc[routes_to_check])

down_time_train        1.666667
interval_giving        0.000000
idle_train             6.333333
travel_time            3.666667
crew_down_time              NaN
crew_down_time_plan         NaN

0.6


In [20]:
classifier.predict(X_valid.iloc[routes_to_check:routes_to_check + 1])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([0.41183331])

In [21]:
r2_score(y_valid, preds)

0.5182479123459905